In [1]:
#https://github.com/garywu/google-compute-engine-selenium
# install chromium, its driver, and selenium
# !sudo apt install chromium-chromedriver
# !sudo cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',options=options)


# open it, go to a website, and get results
# wd = webdriver.Chrome('chromedriver',options=options)
# wd.get("https://www.website.com")
# print(wd.page_source)  # results

In [2]:
import csv


f = open("boozt.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","size","price","color","url","img_url","article_no","blob_url"])
writer.writeheader()
f.close()

import pandas as pd
df=pd.read_csv("boozt.csv", encoding='utf-8')
df.head()

,name,brand,gender,category,size,price,color,url,img_url,article_no,blob_url


In [2]:
cg_list=['overdelar','nederdelar','klanningar-kjolar','badklaeder','bodies','bodysuits','underklaeder-nattklaeder','strumpor-tights','joggingset',
         'basics/overdelar','basics/nederdelar','basics/bodies','activewear/overdelar','activewear/nederdelar','activewear/skor',
         'activewear/accessoarer','activewear/ytterklader','activewear/badklaeder','skor','accessoarer'
        ]


In [3]:
cg_list.pop(0)
cg_list.pop(0)

'nederdelar'

In [4]:
!mkdir boozt

mkdir: cannot create directory ‘boozt’: File exists


In [4]:
import json
import requests
import time
from random import randint
from bs4 import BeautifulSoup
import urllib.request
import csv

for cg in cg_list:
    num=1
    category=cg

    main_url='https://www.boozt.com/se/sv/barn/'+category

    page_urls=main_url+'?page='+str(num)+'&limit=120&grid=small&gender=M'
    gender='boy'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.80 Safari/537.36'}

    driver= webdriver.Chrome('chromedriver',options=options)
    driver.get(page_urls)
    driver.set_window_size(1920,1080)
    
    

    try:
        elem=driver.find_element_by_xpath('//div[@class="dropdown pagination__dropdown dropdown--pagination-desktop"]')
        elem2=elem.find_element_by_tag_name('button')

        driver.execute_script("$(arguments[0]).click();", elem2)
        time.sleep(3)
        p_list=driver.find_element_by_xpath('//div[@class="dropdown__list"]').text
        pages=p_list.split('\n')
    except:
        pages=[1]
    driver.quit()
    print(pages)
    for pu in pages:
        num=pu
        page_url=main_url+'?page='+str(num)+'&limit=120&grid=small&gender=M'
        print(page_url)

        html_doc=requests.get(page_url).content.decode()
        soup = BeautifulSoup(html_doc, 'html.parser')
        products=soup.find_all("li",attrs={"data-group":"listing"})
        for ps in products:
            try:
                url='https://www.boozt.com'+ps.div.a['href']

                article_no=ps['data-product-id']
                price=ps['data-actual-price']
                name=ps['data-product-name']
                brand=soup.find("span",attrs={"class":"p-card__brand"}).text


                html_doc2=requests.get(url).content.decode()
                soup2 = BeautifulSoup(html_doc2, 'html.parser')
                color=soup2.find("span",attrs={"id":"pcolor"}).text
                try:
                    composition=soup2.find("div",attrs={"class":"product-information__material"}).text
                except:
                    composition='None'

                img_url=soup2.find("div",attrs={"class":"product-images"}).find('img')['src']
                ac='boozt'+'/'+article_no+str('.jpg')
        #             opener = urllib.request.URLopener()
        #             opener.addheader('User-Agent', 'whatever')
        #             filename, headers2 = opener.retrieve(img_url, ac)

                rr = requests.get(img_url, allow_redirects=True)
                open(ac, 'wb').write(rr.content)

                sizes=soup2.find("ul",attrs={"class":"our-sizes"})

                for sz in sizes.find_all("li"):
                    size=sz.text
        #             print(name,brand,gender,category,size,price,color,url,img_url,article_no)
                    with open("boozt.csv", "a", encoding='utf-8') as f:
                                writer = csv.writer(f)
                                writer.writerow([name,brand,gender,category,size,price,color,url,img_url,article_no])

#                 time.sleep(randint(1,4))
            except:
                continue

        



[1]
https://www.boozt.com/se/sv/barn/klanningar-kjolar?page=1&limit=120&grid=small&gender=M
['1', '2']
https://www.boozt.com/se/sv/barn/badklaeder?page=1&limit=120&grid=small&gender=M
https://www.boozt.com/se/sv/barn/badklaeder?page=2&limit=120&grid=small&gender=M
['1', '2', '3']
https://www.boozt.com/se/sv/barn/bodies?page=1&limit=120&grid=small&gender=M
https://www.boozt.com/se/sv/barn/bodies?page=2&limit=120&grid=small&gender=M
https://www.boozt.com/se/sv/barn/bodies?page=3&limit=120&grid=small&gender=M
[1]
https://www.boozt.com/se/sv/barn/bodysuits?page=1&limit=120&grid=small&gender=M
['1', '2']
https://www.boozt.com/se/sv/barn/underklaeder-nattklaeder?page=1&limit=120&grid=small&gender=M
https://www.boozt.com/se/sv/barn/underklaeder-nattklaeder?page=2&limit=120&grid=small&gender=M
['1', '2', '3']
https://www.boozt.com/se/sv/barn/strumpor-tights?page=1&limit=120&grid=small&gender=M
https://www.boozt.com/se/sv/barn/strumpor-tights?page=2&limit=120&grid=small&gender=M
https://www.boo

In [5]:
cg_list=['overdelar','nederdelar','klanningar-kjolar','badklaeder','bodies','bodysuits','underklaeder-nattklaeder','strumpor-tights','joggingset',
         'basics/overdelar','basics/nederdelar','basics/bodies','activewear/overdelar','activewear/nederdelar','activewear/skor',
         'activewear/accessoarer','activewear/ytterklader','activewear/badklaeder','skor','accessoarer'
        ]


In [ ]:
import json
import requests
import time
from random import randint
from bs4 import BeautifulSoup
import urllib.request
import csv

for cg in cg_list:
    num=1
    category=cg

    main_url='https://www.boozt.com/se/sv/barn/'+category

    page_urls=main_url+'?page='+str(num)+'&limit=120&grid=small&gender=F'
    gender='girl'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.80 Safari/537.36'}

    driver= webdriver.Chrome('chromedriver',options=options)
    driver.get(page_urls)
    driver.set_window_size(1920,1080)
    try:
        elem=driver.find_element_by_xpath('//div[@class="dropdown pagination__dropdown dropdown--pagination-desktop"]')
        elem2=elem.find_element_by_tag_name('button')

        driver.execute_script("$(arguments[0]).click();", elem2)
        time.sleep(3)

        p_list=driver.find_element_by_xpath('//div[@class="dropdown__list"]').text
        pages=p_list.split('\n')
    except:
        pages=[1]
    driver.quit()
    for pu in pages:
        num=pu
        page_url=main_url+'?page='+str(num)+'&limit=120&grid=small&gender=F'
        print(page_url)

        html_doc=requests.get(page_url).content.decode()
        soup = BeautifulSoup(html_doc, 'html.parser')
        products=soup.find_all("li",attrs={"data-group":"listing"})
        for ps in products:
            try:
                url='https://www.boozt.com'+ps.div.a['href']

                article_no=ps['data-product-id']
                price=ps['data-actual-price']
                name=ps['data-product-name']
                brand=soup.find("span",attrs={"class":"p-card__brand"}).text


                html_doc2=requests.get(url).content.decode()
                soup2 = BeautifulSoup(html_doc2, 'html.parser')
                color=soup2.find("span",attrs={"id":"pcolor"}).text
                try:
                    composition=soup2.find("div",attrs={"class":"product-information__material"}).text
                except:
                    composition='None'


                img_url=soup2.find("div",attrs={"class":"product-images"}).find('img')['src']
                ac='boozt'+'/'+article_no+str('.jpg')
        #             opener = urllib.request.URLopener()
        #             opener.addheader('User-Agent', 'whatever')
        #             filename, headers2 = opener.retrieve(img_url, ac)

                rr = requests.get(img_url, allow_redirects=True)
                open(ac, 'wb').write(rr.content)

                sizes=soup2.find("ul",attrs={"class":"our-sizes"})

                for sz in sizes.find_all("li"):
                    size=sz.text
        #             print(name,brand,gender,category,size,price,color,url,img_url,article_no)
                    with open("boozt.csv", "a", encoding='utf-8') as f:
                                writer = csv.writer(f)
                                writer.writerow([name,brand,gender,category,size,price,color,url,img_url,article_no])

                time.sleep(randint(1,4))
            except:
                continue
        



https://www.boozt.com/se/sv/barn/overdelar?page=1&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=2&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=3&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=4&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=5&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=6&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=7&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=8&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=9&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=10&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=11&limit=120&grid=small&gender=F
https://www.boozt.com/se/sv/barn/overdelar?page=12&limit=120&grid=small&gender=F


In [ ]:
brand

In [ ]:
url